In [22]:
#import Panda dependency
import pandas as pd

In [23]:
#LOADING DATA
#filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

In [24]:
#CLEANING DATA
#clean student names using list, looping through, and using "replace" to change target string into desired value ("") -- blank in this case
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [25]:
#MERGING DATA + METRICS
#merge both dfs for complete data set get basic metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

#pass or fail into new DFs, do counts, and percentages
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

In [26]:
#WRANGLING FOR REQUIREMENT #1

#create district summary df
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

#format each column appropriately
dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.0f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.0f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.0f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

#reorder columns appropriately
col_order = ["Total Schools","Total Students", "Total Budget", "Ave Math Score", 
             "Ave Read Score", "% Pass Math", "% Pass Read", "% Overall Pass"]
dist_sum_df = dist_sum_df[col_order]

#print output
#dist_sum_df

In [27]:
# WRANGLING FOR REQUIREMENT 2

#determine school types
per_school_types = sch_df.set_index(["school_name"])["type"]
per_school_types

#use value_counts for categorical counting
stud_per_school = stu_df["school_name"].value_counts()

#create series for basic budget table and do calculation
budget_per_school = school_types = sch_df.set_index(["school_name"])["budget"]
per_school_capita = budget_per_school/stud_per_school

#grab mean math and read per school
school_math_mean = all_data.groupby(["school_name"]).mean()["math_score"]
school_read_mean = all_data.groupby(["school_name"]).mean()["reading_score"]

#per school pass math/read create DF and count per DF 
school_pass_math = all_data[(all_data["math_score"] > 69)]
school_pass_read = all_data[(all_data["reading_score"] > 69)]
school_pass_overall = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]
school_pass_m_count = school_pass_math.groupby(["school_name"]).count()["student_name"]
school_pass_r_count = school_pass_read.groupby(["school_name"]).count()["student_name"]
school_pass_all_count = school_pass_overall.groupby(["school_name"]).count()["student_name"]

#calculate percent pass per class per school
math_pass_perc_school = school_pass_m_count / stud_per_school * 100
read_pass_perc_school = school_pass_r_count / stud_per_school * 100
overall_pass_perc_school = school_pass_all_count / stud_per_school * 100

#create  and format output DF
school_sum_df = pd.DataFrame({
    "School Type" : per_school_types,
    "Total Students" : stud_per_school,
    "Total School Budget" : budget_per_school,
    "Per Student Budget" : per_school_capita,
    "Ave Math Score" : school_math_mean,
    "Ave Read Score" : school_read_mean,
    "% Pass Math" : math_pass_perc_school,
    "% Pass Read" : read_pass_perc_school,
    "% Overall Pass" : overall_pass_perc_school})


#FORMAT LATER
#this df will be used althroughut for later wrangling

#print output
#school_sum_df

In [28]:
#REQUIREMENT 3.1 Top 5 schools per overall passing

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = False)

#top_sch.head(5)

In [29]:
#REQUIREMENT 3.2 Bottom 5 schools per overall passing

bottom_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = True)

#bottom_sch.head(5)

In [30]:
#REQUIREMENT 3.3 Per School, per Grade, Math and READ Ave Scores

grade9 = all_data[(all_data["grade"]== "9th")]
grade10 = all_data[(all_data["grade"] == "10th" )]
grade11 = all_data[(all_data["grade"] == "11th" )]
grade12 = all_data[(all_data["grade"] == "12th" )]

grade9_math = grade9.groupby(["school_name"]).mean()["math_score"]
grade10_math = grade10.groupby(["school_name"]).mean()["math_score"]
grade11_math = grade11.groupby(["school_name"]).mean()["math_score"]
grade12_math = grade12.groupby(["school_name"]).mean()["math_score"]

grade9_read = grade9.groupby(["school_name"]).mean()["reading_score"]
grade10_read = grade10.groupby(["school_name"]).mean()["reading_score"]
grade11_read = grade11.groupby(["school_name"]).mean()["reading_score"]
grade12_read = grade12.groupby(["school_name"]).mean()["reading_score"]

#create and format math per grade per df
math_per_grade = pd.DataFrame ({
    "9th" : grade9_math,
    "10th" : grade10_math,
    "11th" : grade11_math,
    "12th" : grade12_math})

math_per_grade["9th"] = math_per_grade["9th"].map("{:.1f}".format)
math_per_grade["10th"] = math_per_grade["10th"].map("{:.1f}".format)
math_per_grade["11th"] = math_per_grade["11th"].map("{:.1f}".format)
math_per_grade["12th"] = math_per_grade["12th"].map("{:.1f}".format)

#create and format math per grade per school df
read_per_grade = pd.DataFrame ({
    "9th" : grade9_read,
    "10th" : grade10_read,
    "11th" : grade11_read,
    "12th" : grade12_read})

read_per_grade["9th"] = read_per_grade["9th"].map("{:.1f}".format)
read_per_grade["10th"] = read_per_grade["10th"].map("{:.1f}".format)
read_per_grade["11th"] = read_per_grade["11th"].map("{:.1f}".format)
read_per_grade["12th"] = read_per_grade["12th"].map("{:.1f}".format)

# correct order, remove indices if any
math_per_grade = math_per_grade[["9th", "10th", "11th", "12th"]]
read_per_grade = read_per_grade[["9th", "10th", "11th", "12th"]]
math_per_grade.index.name = None
read_per_grade.index.name = None

#print outputs - TWO OUTPUTS
#math_per_grade
#read_per_grade

In [31]:
#Per SPENDING RANGES as indices, raw and ave math score, read score, and overall score per index

#create bins, labels, and CUT data, add to school summary DF
spending_bins = [0,585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
per_school_capita.groupby(pd.cut(per_school_capita, spend_bins)).count()
school_sum_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

#school summary with spend grouping
spend_math_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Math Score"]
spend_math_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Math"]
spend_read_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Read Score"]
spend_read_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Read"]
spend_overall_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Pass"]

#create and format output df
spend_sum_df = pd.DataFrame({
          "Ave Math Score" : spend_math_grade,
          "Ave Read Score": spend_read_grade,
          "% Pass Math": spend_math_perc,
          "% Pass Read": spend_read_perc,
          "% Overall Pass": spend_overall_perc})

spend_sum_df["Ave Math Score"] = spend_sum_df["Ave Math Score"].map("{:.1f}".format)
spend_sum_df["Ave Read Score"] = spend_sum_df["Ave Read Score"].map("{:.1f}".format)
spend_sum_df["% Pass Math"] = spend_sum_df["% Pass Math"].map("{:.0f}".format)
spend_sum_df["% Pass Read"] = spend_sum_df["% Pass Read"].map("{:.0f}".format)
spend_sum_df["% Overall Pass"] = spend_sum_df["% Overall Pass"].map("{:.0f}".format)

#print output
#spend_sum_df

NameError: name 'spend_bins' is not defined

In [32]:
#Per SCHOOL SIZE as indices, raw and ave math score, read score, and overall score per index

#stud_per_school = all_data["school_name"].value_counts()
#stud_per_school

stud_bins = [0, 1000, 2000, 5000]
bin_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_sum_df["School Size"] = pd.cut(school_sum_df["Total Students"], stud_bins, labels=bin_labels)

size_math_grade = school_sum_df.groupby(["School Size"]).mean()["Ave Math Score"]
size_math_perc = school_sum_df.groupby(["School Size"]).mean()["% Pass Math"]
size_read_grade = school_sum_df.groupby(["School Size"]).mean()["Ave Read Score"]
size_read_perc = school_sum_df.groupby(["School Size"]).mean()["% Pass Read"]
size_overall_perc = school_sum_df.groupby(["School Size"]).mean()["% Overall Pass"]

#create and format output df
size_sum_df = pd.DataFrame({
          "Ave Math Score" : size_math_grade,
          "Ave Read Score": size_read_grade,
          "% Pass Math": size_math_perc,
          "% Pass Read": size_read_perc,
          "% Overall Pass": size_overall_perc})

size_sum_df["Ave Math Score"] = size_sum_df["Ave Math Score"].map("{:.1f}".format)
size_sum_df["Ave Read Score"] = size_sum_df["Ave Read Score"].map("{:.1f}".format)
size_sum_df["% Pass Math"] = size_sum_df["% Pass Math"].map("{:.0f}".format)
size_sum_df["% Pass Read"] = size_sum_df["% Pass Read"].map("{:.0f}".format)
size_sum_df["% Overall Pass"] = size_sum_df["% Overall Pass"].map("{:.0f}".format)

#print output
#size_sum_df

In [33]:
#Per SCHOOL TYPE as indices, raw and ave math score, read score, and overall score per index

type_math_grade = school_sum_df.groupby(["School Type"]).mean()["Ave Math Score"]
type_math_perc = school_sum_df.groupby(["School Type"]).mean()["% Pass Math"]
type_read_grade = school_sum_df.groupby(["School Type"]).mean()["Ave Read Score"]
type_read_perc = school_sum_df.groupby(["School Type"]).mean()["% Pass Read"]
type_overall_perc = school_sum_df.groupby(["School Type"]).mean()["% Overall Pass"]

#create and format output df
type_sum_df = pd.DataFrame({
          "Ave Math Score" : type_math_grade,
          "Ave Read Score": type_read_grade,
          "% Pass Math": type_math_perc,
          "% Pass Read": type_read_perc,
          "% Overall Pass": type_overall_perc})

type_sum_df["Ave Math Score"] = type_sum_df["Ave Math Score"].map("{:.1f}".format)
type_sum_df["Ave Read Score"] = type_sum_df["Ave Read Score"].map("{:.1f}".format)
type_sum_df["% Pass Math"] = type_sum_df["% Pass Math"].map("{:.0f}".format)
type_sum_df["% Pass Read"] = type_sum_df["% Pass Read"].map("{:.0f}".format)
type_sum_df["% Overall Pass"] = type_sum_df["% Overall Pass"].map("{:.0f}".format)

#print output
#type_sum_df

In [34]:
#Requirement 1 - District Key Metrics
dist_sum_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [35]:
#Requirement 2 - Per School Metrics

#formatting now -- because school summary DF was being used althroughout other requirements
school_sum_df["Total Students"] = school_sum_df["Total Students"].map("{:,}".format)
school_sum_df["Ave Math Score"] = school_sum_df["Ave Math Score"].map("{:.1f}".format)
school_sum_df["Ave Read Score"] = school_sum_df["Ave Read Score"].map("{:.1f}".format)
school_sum_df["% Pass Math"] = school_sum_df["% Pass Math"].map("{:.2f}".format)
school_sum_df["% Pass Read"] = school_sum_df["% Pass Read"].map("{:.2f}".format)
school_sum_df["% Overall Pass"] = school_sum_df["% Overall Pass"].map("{:.2f}".format)
school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)
school_sum_df["Per Student Budget"] = school_sum_df["Per Student Budget"].map("${:,.2f}".format)

school_sum_df


,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass,School Size
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0,81.0,66.68,81.93,54.64,Large (2000-5000)
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.13,97.04,91.33,Medium (1000-2000)
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,65.99,80.74,53.20,Large (2000-5000)
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68.31,79.30,54.29,Large (2000-5000)
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.39,97.14,90.60,Medium (1000-2000)
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,66.75,80.86,53.53,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,92.51,96.25,89.23,Small (<1000)
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.68,81.32,53.51,Large (2000-5000)
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81.0,66.06,81.22,53.54,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.59,95.95,90.54,Small (<1000)


In [36]:
#Requirement 3.A.i - Top 5 - Per Pass Rate
top_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [37]:
#Requirement 3.A.ii - Bottom 5 - Per Pass Rate
bottom_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [38]:
#Requirement 3.B.i - Ave Math Scores per Grade
math_per_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [39]:
#Requirement 3.B.ii - Ave Read Scores per Grade
read_per_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


In [40]:
#Requirement 3.C.i - Grades and Pass Rate per Spend Category
spend_sum_df

NameError: name 'spend_sum_df' is not defined

In [41]:
#Requirement 3.C.i - Grades and Pass Rate per School Size
size_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58


In [42]:
#Requirement 3.C.iii - Grades and Pass Rate per School Type
type_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
